In [ ]:
import numpy as np
import h5py
import matplotlib.pyplot as plt

import fast_body_tracker as fbt

%matplotlib notebook

In [ ]:
file_path = "../data/analysis_data/body.h5"
with h5py.File(file_path, "r") as f:
    ts = []
    frame_idx = []
    positions = []
    ts_grp = f["ts"]
    joints_grp = f["joints"]
    for device_key in ts_grp.keys():
        device_grp = ts_grp[device_key]
        ts.append(device_grp["ts"][:].astype(np.int64))

        body_grp = joints_grp[device_key]["body_0"]
        frame_idx.append(body_grp["frame_idx"][:])
        positions.append(body_grp["positions"][:])

In [ ]:
frame_idx0 = frame_idx[0]
ts0 = ts[0]

valid_mask = np.ones(frame_idx0.size, dtype=bool)
aligned_frame_idx = {0: frame_idx0}
for i in range(1, len(ts)):
    tsi = ts[i]

    frame_idxi = frame_idx[i]
    idxi = np.searchsorted(tsi, ts0)
    idxi = np.clip(idxi, 1, tsi.size-1)

    dist_right = np.abs(tsi[idxi] - ts0)
    dist_left = np.abs(tsi[idxi-1] - ts0)
    closer_to_left = dist_left < dist_right
    idxi[closer_to_left] -= 1

    aligned_frame_idxi = idxi[frame_idx0]
    aligned_frame_idx[i] = aligned_frame_idxi

    has_body = np.isin(aligned_frame_idxi, frame_idxi)
    valid_mask &= has_body
aligned_frame_idx = {i: f_idx[valid_mask] for i, f_idx in aligned_frame_idx.items()}

print(aligned_frame_idx[0].size)

In [ ]:
ref_frame_rank = 2200

colors = plt.get_cmap("tab10").colors
fig = plt.figure()
ax = fig.add_subplot(projection="3d")
for i, aligned_frame_idxi in aligned_frame_idx.items():
    idx = aligned_frame_idxi[ref_frame_rank]
    idx = np.where(frame_idx[i] == idx)[0][0]
    pos = positions[i][idx]

    c = colors[i % 10]
    ax.scatter(pos[:, 0], pos[:, 2], - pos[:, 1], color=c, label=f"Device {i}")
    for start, end in fbt.K4ABT_SEGMENT_PAIRS:
        ax.plot(pos[[start, end], 0], pos[[start, end], 2], - pos[[start, end], 1], color=c)

ax.set_xlabel("x"); ax.set_ylabel("z"); ax.set_zlabel("y")
ax.set_title("Spatial calibration")
ax.legend()
plt.show()